# SI 618 Fall 2022 Homework 3 (100 points)

- Name: Sijun Tao
- Uniquename: sijuntao

## Part 1 (80 points) 

The provided ‘movie_actors_data.txt’ file contains a JSON string on each line. For example, the first line is: 

>{"rating": 9.3, "genres": ["Crime", "Drama"], "rated": "R", "filming_locations": "Ashland, Ohio, USA", "language": ["English"], "title": "The Shawshank Redemption", "runtime": ["142 min"], "poster": "http://img3.douban.com/lpic/s1311361.jpg", "imdb_url": "http://www.imdb.com/title/tt0111161/", "writers": ["Stephen King", "Frank Darabont"], "imdb_id": "tt0111161", "directors": ["Frank Darabont"], "rating_count": 894012, "actors": ["Tim Robbins", "Morgan Freeman", "Bob Gunton", "William Sadler", "Clancy Brown", "Gil Bellows", "Mark Rolston", "James Whitmore", "Jeffrey DeMunn", "Larry Brandenburg", "Neil Giuntoli", "Brian Libby", "David Proval", "Joseph Ragno", "Jude Ciccolella"], "plot_simple": "Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.", "year": 1994, "country": ["USA"], "type": "M", "release_date": 19941014, "also_known_as": ["Die Verurteilten"]}

The fields we are interested in are imdb_id, title , rating, genres, actors, directors, year, and country. You will parse the JSON strings, and load the data into three tables in SQLite, and then write SQL queries to retrieve the data specified. 

You will create 4 tables: 
* The “movies” table, which has four columns: imdb_id, title, year, rating, country  

* The “movie_genre” table, which has two columns: imdb_id and genre. A movie typically has multiple genres, and in this case, there should be one row for each genre. If some movie does not have any genre, ignore that movie.  

* The “movie_actor” table, which has two columns imdb_id and actor. A movie typically has multiple actors, and in this case, there should be one row for each actor. 

* The “movie_director” table, which has two columns imdb_id and director. A movie typically has multiple directors, and in this case, there should be one row for each director. 

**When you run your Python code, it should print out EXACTLY the output provided.**

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import json
import urllib3
import sqlite3
conn = sqlite3.connect('sijuntao_si618_h3.db')
urllib3.disable_warnings()

#### Q1. (10 points) Parse input file to get needed data for the three tables and load them into appropriate Python data structure. 

In [2]:
file = open("movie_actors_data.txt", "r")
data = file.readlines()
file.close()

data = [json.loads(row.replace('\n','')) for row in data]
df = pd.DataFrame(data)
df.head()

,rating,genres,rated,filming_locations,language,title,runtime,poster,imdb_url,writers,imdb_id,directors,rating_count,actors,plot_simple,year,country,type,release_date,also_known_as
0,9.3,"[Crime, Drama]",R,"Ashland, Ohio, USA",[English],The Shawshank Redemption,[142 min],http://img3.douban.com/lpic/s1311361.jpg,http://www.imdb.com/title/tt0111161/,"[Stephen King, Frank Darabont]",tt0111161,[Frank Darabont],894012,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will...",Two imprisoned men bond over a number of years...,1994,[USA],M,19941014,[Die Verurteilten]
1,9.2,"[Crime, Drama]",R,"110 Longfellow Road, Staten Island, New York C...","[English, Italian, Latin]",The Godfather,[175 min],http://img3.douban.com/lpic/s4038344.jpg,http://www.imdb.com/title/tt0068646/,"[Mario Puzo, Francis Ford Coppola, and 1 more ...",tt0068646,[Francis Ford Coppola],646348,"[Marlon Brando, Al Pacino, James Caan, Richard...",The aging patriarch of an organized crime dyna...,1972,[USA],M,19720324,[Mario Puzo's The Godfather]
2,9.1,"[Crime, Drama]",R,"6th Street, Manhattan, New York City, New York...","[English, Italian, Spanish, Latin]",The Godfather: Part II,[200 min],http://img3.douban.com/lpic/s3314652.jpg,http://www.imdb.com/title/tt0071562/,"[Francis Ford Coppola, Mario Puzo, and 1 more ...",tt0071562,[Francis Ford Coppola],415305,"[Al Pacino, Robert Duvall, Diane Keaton, Rober...",The early life and career of Vito Corleone in ...,1974,[USA],M,19741220,[Mario Puzo's The Godfather: Part II]
3,9.0,"[Crime, Thriller]",R,"1525 N. Van Ness Avenue, Los Angeles, Californ...","[English, Spanish, French]",Pulp Fiction,"[154 min, USA: 168 min (special edition)]",http://img3.douban.com/lpic/s2845185.jpg,http://www.imdb.com/title/tt0110912/,"[Quentin Tarantino, Roger Avary, and 1 more cr...",tt0110912,[Quentin Tarantino],694866,"[John Travolta, Samuel L. Jackson, Tim Roth, A...","The lives of two mob hit men, a boxer, a gangs...",1994,[USA],M,19941014,[Tiempos violentos]
4,9.0,"[Adventure, Western]",APPROVED,"Almería, Andalucía, Spain",[Italian],"Il buono, il brutto, il cattivo.","[161 min, France: 186 min (dubbed version), Sp...",http://img3.douban.com/lpic/s4008151.jpg,http://www.imdb.com/title/tt0060196/,"[Luciano Vincenzoni, Sergio Leone]",tt0060196,[Sergio Leone],274781,"[Eli Wallach, Clint Eastwood, Lee Van Cleef, A...",A bounty hunting scam joins two men in an unea...,1966,"[Italy, Spain, West Germany]",M,19671229,"[The Good, the Bad and the Ugly]"


#### Q2. (5 points) Create the movies table and load data into it. Note that you only need to consider the first country if there are multiple countries associated with a movie. Read and show the first 5 rows of the movies table in your SQLite database.

In [3]:
c = conn.cursor()
c.execute('CREATE TABLE movies (imdb_id, title, year, rating, country )')
movies_data = []
for i in range(0,len(df)):
    movies_data.append((df.loc[i,"imdb_id"], df.loc[i, "title"], int(df.loc[i, "year"]), df.loc[i, "rating"], df.loc[i, "country"][0]))

c.executemany('INSERT INTO movies VALUES (?,?,?,?,?)', movies_data)
conn.commit()

In [4]:
movies_first_5 = c.execute('SELECT * FROM movies LIMIT 5')
for row in movies_first_5:
    print (row)

('tt0111161', 'The Shawshank Redemption', 1994, 9.3, 'USA')
('tt0068646', 'The Godfather', 1972, 9.2, 'USA')
('tt0071562', 'The Godfather: Part II', 1974, 9.1, 'USA')
('tt0110912', 'Pulp Fiction', 1994, 9.0, 'USA')
('tt0060196', 'Il buono, il brutto, il cattivo.', 1966, 9.0, 'Italy')


#### Q3. (5 points) Create the movie_genre table and load data into it.

In [5]:
c.execute('CREATE TABLE movie_genre (imdb_id, genre)')

movie_genre_df = df[["imdb_id", "genres"]].explode("genres").reset_index()
movie_genre_data = []
for i in range(0,len(movie_genre_df)):
    movie_genre_data.append((movie_genre_df.loc[i,"imdb_id"], movie_genre_df.loc[i, "genres"]))

c.executemany('INSERT INTO movie_genre VALUES (?,?)', movie_genre_data)
conn.commit()

In [6]:
movie_genre_first_5 = c.execute('SELECT * FROM movie_genre LIMIT 5')
for row in movie_genre_first_5:
    print (row)

('tt0111161', 'Crime')
('tt0111161', 'Drama')
('tt0068646', 'Crime')
('tt0068646', 'Drama')
('tt0071562', 'Crime')


#### Q4. (5 points) Create the movie_actor table and load data into it.

In [7]:
c.execute('CREATE TABLE movie_actor (imdb_id, actor)')

movie_actor_df = df[["imdb_id", "actors"]].explode("actors").reset_index()
movie_actor_data = []
for i in range(0,len(movie_actor_df)):
    movie_actor_data.append((movie_actor_df.loc[i,"imdb_id"], movie_actor_df.loc[i, "actors"]))

c.executemany('INSERT INTO movie_actor VALUES (?,?)', movie_actor_data)
conn.commit()

In [8]:
movie_actor_first_5 = c.execute('SELECT * FROM movie_actor LIMIT 5')
for row in movie_actor_first_5:
    print (row)

('tt0111161', 'Tim Robbins')
('tt0111161', 'Morgan Freeman')
('tt0111161', 'Bob Gunton')
('tt0111161', 'William Sadler')
('tt0111161', 'Clancy Brown')


#### Q5. (5 points) Create the movie_director table and load data into it.

In [9]:
c.execute('CREATE TABLE movie_director (imdb_id, director)')

movie_director_df = df[["imdb_id", "directors"]].explode("directors").reset_index()
movie_director_data = []
for i in range(0,len(movie_director_df)):
    movie_director_data.append((movie_director_df.loc[i,"imdb_id"], movie_director_df.loc[i, "directors"]))

c.executemany('INSERT INTO movie_director VALUES (?,?)', movie_director_data)
conn.commit()

In [10]:
movie_director_first_5 = c.execute('SELECT * FROM movie_director LIMIT 5')
for row in movie_director_first_5:
    print (row)

('tt0111161', 'Frank Darabont')
('tt0068646', 'Francis Ford Coppola')
('tt0071562', 'Francis Ford Coppola')
('tt0110912', 'Quentin Tarantino')
('tt0060196', 'Sergio Leone')


#### Q6. (10 points) Write a SQL query to find the number of released movies by decade in chronological order. (Note: you will need to round *down* years to get the corresponding decade. e.g. 2009 belongs to the 2000s)
```
Number of movies broken down by decade: Decade, Number
1920, 5
1930, 8
1940, 16
1950, 30
1960, 23
1970, 24
1980, 26
1990, 38
2000, 57
2010, 21

```

In [11]:
command = "SELECT (year/10)*10, COUNT(*) " \
          "FROM movies " \
          "GROUP BY (year/10)*10 "  
results = c.execute(command)

print("Number of movies broken down by decade: Decade, Number")
for row in results:
    print(f"{row[0]}, {row[1]}")

Number of movies broken down by decade: Decade, Number
1920, 5
1930, 8
1940, 16
1950, 30
1960, 23
1970, 24
1980, 26
1990, 38
2000, 57
2010, 21


#### Q7. (10 points) Write a SQL query to find the top 5 genres in the US by the average ratings of movies in that genre. Print out the results.
```
Top 5 genres in the US: Genre, AvgRatings
Action, 8.461290322580647
War, 8.413333333333332
Crime, 8.408695652173916
Music, 8.4
Sci-Fi, 8.385000000000002
```

In [12]:
command = "SELECT genre, AVG(rating) " \
          "FROM (movie_genre JOIN movies on movies.imdb_id = movie_genre.imdb_id) " \
          "WHERE country == 'USA' " \
          "GROUP BY genre " \
          "ORDER BY AVG(rating) DESC " \
          "LIMIT 5"
results = c.execute(command)

print("Top 5 genres in the US: Genre, AvgRatings")
for row in results:
    print(f"{row[0]}, {row[1]}")

Top 5 genres in the US: Genre, AvgRatings
Action, 8.461290322580647
War, 8.413333333333332
Crime, 8.408695652173916
Music, 8.4
Sci-Fi, 8.385000000000002


#### Q8. (10 points) Write a SQL query to find the top 10 actors who have worked with the director Christopher Nolan in the most movies. The actors are ordered in decreasing number of movies they worked with the director. In case of ties, the rows are ordered by actors’ names in alphabetical order.

```
The actors who have worked with Christopher Nolan the most: Name, Times
Michael Caine, 5
Christian Bale, 4
Cillian Murphy, 3
Gary Oldman, 3
Morgan Freeman, 3
Joseph Gordon-Levitt, 2
Ken Watanabe, 2
Larry Holden, 2
Marion Cotillard, 2
Mark Boone Junior, 2
```

In [13]:
command = "SELECT actor, COUNT(actor) " \
          "FROM (movie_actor JOIN movie_director on movie_actor.imdb_id = movie_director.imdb_id) " \
          "WHERE director == 'Christopher Nolan' " \
          "GROUP BY actor " \
          "ORDER BY COUNT(actor) DESC, actor " \
          "LIMIT 10"
results = c.execute(command)

print("The actors who have worked with Christopher Nolan the most: Name, Times")
for row in results:
    print(f"{row[0]}, {row[1]}")

The actors who have worked with Christopher Nolan the most: Name, Times
Michael Caine, 5
Christian Bale, 4
Cillian Murphy, 3
Gary Oldman, 3
Morgan Freeman, 3
Joseph Gordon-Levitt, 2
Ken Watanabe, 2
Larry Holden, 2
Marion Cotillard, 2
Mark Boone Junior, 2


#### Q9. (10 points) Write a SQL query to find the top 10 directors based on average ratings of the movies they directed between the years year 1980 and 2010 (begin and end years are included). Only consider those who directed at least 2 movies in this time range. For each director, show their name, average rating of the movies, and the number of movies they directed. Sort the result by descending average movie rating, then by director name in alphabetical order.

```
Between 1980 and 2010, top 10 highest rated directors based on average movie rating with at least 2 credits:
Director, Movies
Frank Darabont, 8.9, 2
David Fincher, 8.8, 2
Peter Jackson, 8.8, 3
Christopher Nolan, 8.64, 5
Robert Zemeckis, 8.6, 2
Steven Spielberg, 8.5, 5
Quentin Tarantino, 8.475, 4
Stanley Kubrick, 8.45, 2
James Cameron, 8.4, 3
Ridley Scott, 8.4, 2
```


In [14]:
command = "SELECT director, AVG(rating), COUNT(*) " \
          "FROM (movie_director JOIN movies on movies.imdb_id = movie_director.imdb_id) " \
          "WHERE year>=1980 AND year<=2010 " \
          "GROUP BY director " \
          "HAVING COUNT(*)>=2 " \
          "ORDER BY AVG(rating) DESC, director ASC " \
          "LIMIT 10"
results = c.execute(command)

print("Between 1980 and 2010, top 10 highest rated directors based on average movie rating with at least 2 credits: Director, Movies")
for row in results:
    print(f"{row[0]}, {row[1]}, {row[2]}")

Between 1980 and 2010, top 10 highest rated directors based on average movie rating with at least 2 credits: Director, Movies
Frank Darabont, 8.9, 2
David Fincher, 8.8, 2
Peter Jackson, 8.8, 3
Christopher Nolan, 8.64, 5
Robert Zemeckis, 8.6, 2
Steven Spielberg, 8.5, 5
Quentin Tarantino, 8.475, 4
Stanley Kubrick, 8.45, 2
James Cameron, 8.4, 3
Ridley Scott, 8.4, 2


#### Q10. (10 points) Write a SQL query for finding pairs of actors who co-starred in at least 2 highly rated (rating > 9) movies together. 

The pairs of names must be unique. This means that ‘actor A, actor B’ and ‘actor B, actor A’ are the same pair, so only one of them should appear. For each pair of actors you print out, the two actors must be ordered alphabetically. The pairs are ordered in decreasing number of movies they co-stared in. In case of ties, the rows are ordered by actors’ names. 

>You will need to join the movie_actor table with itself to get this data (in addition to another required join). It is a bit tricky. If you cannot do it with SQL statement, you can also write some Python code that works on the Python data structure that you used to create the movie_actor table. That’ll mean much more lines of code, and if you do it that way, you’ll get 5 points instead of 10 points. You will only get 10 points if you solve it with pure SQL.

```
Pairs of actors who co-stared in 2 or more highly rated movies: Actor A, Actor B, Co-stared Movies
Al Pacino, Diane Keaton, 2
Al Pacino, John Cazale, 2
Al Pacino, Robert Duvall, 2
Al Pacino, Talia Shire, 2
Diane Keaton, John Cazale, 2
Diane Keaton, Robert Duvall, 2
Diane Keaton, Talia Shire, 2
John Cazale, Robert Duvall, 2
John Cazale, Talia Shire, 2
Robert Duvall, Talia Shire, 2
```

In [15]:
command = "SELECT actor1, actor2, COUNT(*) " \
          "FROM " \
              "(SELECT movie_actor1.imdb_id, movie_actor1.actor AS actor1, movie_actor2.actor AS actor2 " \
              "FROM (movie_actor movie_actor1, movie_actor movie_actor2) " \
              "WHERE movie_actor1.imdb_id == movie_actor2.imdb_id AND actor1 < actor2) " \
              "AS co_stared " \
          "JOIN movies on movies.imdb_id = co_stared.imdb_id " \
          "WHERE rating > 9 " \
          "GROUP BY actor1, actor2 " \
          "HAVING COUNT(*) >= 2 " \
          "ORDER BY COUNT(*) DESC, actor1, actor2 "
results = c.execute(command)

print("Pairs of actors who co-stared in 2 or more highly rated movies: Actor A, Actor B, Co-stared Movies")
for row in results:
    print(f"{row[0]}, {row[1]}, {row[2]}")

Pairs of actors who co-stared in 2 or more highly rated movies: Actor A, Actor B, Co-stared Movies
Al Pacino, Diane Keaton, 2
Al Pacino, John Cazale, 2
Al Pacino, Robert Duvall, 2
Al Pacino, Talia Shire, 2
Diane Keaton, John Cazale, 2
Diane Keaton, Robert Duvall, 2
Diane Keaton, Talia Shire, 2
John Cazale, Robert Duvall, 2
John Cazale, Talia Shire, 2
Robert Duvall, Talia Shire, 2


## Part 2 (20 points) 

Write a python function that takes two arguments: genre and k 

The function should print out the top k directors who made the highest rated movies (on average) in the provided genre. You should use the sqlite3 database file you created in Part 1. Some example runs of my function are shown below. Your program should produce the same output when provided with the same command line arguments.

`get_top_genre_directors("Sci-Fi", 3)`
```
Top 3 directors who made highest rated Sci-Fi movies:
Director, Average Sci-Fi movie rating
Christopher Nolan, 8.8
George Lucas, 8.8
Irvin Kershner, 8.8
```

`get_top_genre_directors("War", 5)`
```
Top 5 directors who made highest rated War movies:
Director, Average War movie rating
Steven Spielberg, 8.75
Michael Curtiz, 8.7
Francis Ford Coppola, 8.6
Charles Chaplin, 8.5
Roberto Benigni, 8.5
```

`get_top_genre_directors("Animation", 10)`
```
Top 10 directors who made highest rated Animation movies:
Director, Average Animation movie rating
Isao Takahata, 8.4
Rob Minkoff, 8.4
Roger Allers, 8.4
Andrew Stanton, 8.3
Bob Peterson, 8.3
John Lasseter, 8.3
Lee Unkrich, 8.3
Adam Elliot, 8.2
Chris Sanders, 8.2
Dean DeBlois, 8.2
```

In [16]:
def get_top_genre_directors(genre, k):
    command = "SELECT director, AVG(rating) " \
              "FROM " \
                  "(SELECT movie_director.imdb_id, director, genre " \
                  "FROM (movie_director JOIN movie_genre on movie_director.imdb_id == movie_genre.imdb_id)) " \
                  "AS director_genre " \
              "JOIN movies on movies.imdb_id = director_genre.imdb_id " \
              "WHERE genre == " + "'" + genre + "'" \
              " GROUP BY director " \
              "ORDER BY AVG(rating) DESC, director " \
              "LIMIT " + str(k)
    results = c.execute(command)

    print(f"Top {k} directors who made highest rated "+ genre+" movies:\nDirector, Average " + genre + " movie rating")
    for row in results:
        print(f"{row[0]}, {row[1]}")

In [17]:
get_top_genre_directors("Sci-Fi", 3)

Top 3 directors who made highest rated Sci-Fi movies:
Director, Average Sci-Fi movie rating
Christopher Nolan, 8.8
George Lucas, 8.8
Irvin Kershner, 8.8


In [18]:
get_top_genre_directors("War", 5)

Top 5 directors who made highest rated War movies:
Director, Average War movie rating
Steven Spielberg, 8.75
Michael Curtiz, 8.7
Francis Ford Coppola, 8.6
Charles Chaplin, 8.5
Roberto Benigni, 8.5


In [19]:
get_top_genre_directors("Animation", 10)

Top 10 directors who made highest rated Animation movies:
Director, Average Animation movie rating
Isao Takahata, 8.4
Rob Minkoff, 8.4
Roger Allers, 8.4
Andrew Stanton, 8.3
Bob Peterson, 8.3
John Lasseter, 8.3
Lee Unkrich, 8.3
Adam Elliot, 8.2
Chris Sanders, 8.2
Dean DeBlois, 8.2


## What to submit: 
Please perform the following operations and turn in the corresponding files to canvas:
1. Jupyter notebook titled uniqname_si618_hw3.ipynb and the corresponding html page (uniqname_si618_hw3.html) 
through Canvas.
2. The sqlite database file you created. (uniqname_si618_hw3.db)

In [20]:
conn.close()